## Example of mBuild + Foyer + MD Workflow

First, we'll import the mBuild package (and set a filter for some annoying warnings).

In [ ]:
import mbuild as mb
import warnings
warnings.filterwarnings("ignore")

We'll define an empty `Compound` to serve as a container for the various components in our system.

In [ ]:
system = mb.Compound()

mBuild features a `Lattice` class, that can be used to create, well, lattices. Here, we'll use this to create an FCC lattice of gold.

In [ ]:
gold = mb.Compound(name='Au')
lattice_vectors = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
spacing = [.40782, .40782, .40782]
locations = [[0., 0., 0.], [.5, .5, 0.],
             [.5, 0., .5], [0., .5, .5]]
basis = {'Au': locations}
lattice = mb.Lattice(lattice_spacing=spacing,
                     lattice_vectors=lattice_vectors,
                     lattice_points=basis)
surface = lattice.populate(x=6, y=6, z=2,
                           compound_dict={'Au': gold})
system.add(surface, 'surface')
system.visualize()

Create a layer of eight water molecules to place on top of the gold surface

In [ ]:
from mbuild.lib.moieties import H2O

surface_dimensions = surface.boundingbox
solvent_layer = mb.Box(mins=[surface_dimensions.mins[0],
                             surface_dimensions.mins[1],
                             surface_dimensions.maxs[2] + 0.4],
                       maxs=[surface_dimensions.maxs[0],
                             surface_dimensions.maxs[1],
                             surface_dimensions.maxs[2] + 0.6])
water_layer = mb.fill_box(compound=H2O(), n_compounds=8,
                          box=solvent_layer, seed=12)
system.add(water_layer, 'water')
system.visualize()

Save the system as a LAMMPS data file. LAMMPS is one of the most popular simulation packages, and we will revisit it for our workshop session on Friday.

When going from an mBuild `Compound` to a data file that can be used to perform a simulation, we need to provide the proper force field parameters. This is where Foyer come in. By passing a Foyer force field XML file (or in this case, a list of XML files) to mBuild's `save` function, we are able to use Foyer to perform atomtyping under the hood and assign the proper force field parameters.

In [ ]:
system.periodicity = surface.periodicity + [0, 0, 10]
system.save('gold-h2o.lammps', forcefield_files=['utils/tip3p.xml', 'utils/heinz2008.xml'],
            overwrite=True)

Run a short LAMMPS simulation. (This is just for demo purposes, so don't be too concerned with the input/output. By the end of this course you will be able to put together a simulation script yourself!)

In [ ]:
! lmp_serial -in utils/in.short-eq -echo screen
! python utils/xyz_to_pdb.py

Read the output from the simulation back into mBuild.

In [ ]:
system.update_coordinates('output.pdb')
system.visualize()

Expand the system in the surface plane using mBuild's `TiledCompound` class.

In [ ]:
expanded_system = mb.TiledCompound(system, n_tiles=[5, 5, 1])
expanded_system.visualize()